In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

tickers = {}
ticker_names = ["BTC-USD", "SOL-USD", "ETH-USD", "GC=F", "SI=F"]
for ticker in ticker_names:
    tickers[ticker] = yf.download(ticker, start='2019-12-01', end='2024-11-08')
    tickers[ticker].reset_index(inplace=True)
    relativeChange = tickers[ticker].Close.pct_change(30)*100
    tickers[ticker]['RelativeChange'] = relativeChange
    tickers[ticker] = tickers[ticker].dropna() 
    tickers[ticker]['Legend_label'] = ticker
    tickers[ticker] = tickers[ticker].drop(['Adj Close', 'High', 'Low', 'Open', 'Volume'], axis=1)
    tickers[ticker].columns = tickers[ticker].columns.get_level_values(0)


print(tickers['BTC-USD'])

[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_81090/8477623.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  tickers[ticker] = tickers[ticker].drop(['Adj Close', 'High', 'Low', 'Open', 'Volume'], axis=1)
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_81090/8477623.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  tickers[ticker] = tickers[ticker].drop(['Adj Close', 'High', 'Low', 'Open', 'Volume'], axis=1)
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_81090/8477623.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  tickers[ticker] = tickers[ticker].drop(['Adj Close', 'High', 'Low', 'Open', 'Volume'], axis=1)
[*********************100%***********************]  1 of 1 compl

Price                      Date         Close  RelativeChange Legend_label
30    2019-12-31 00:00:00+00:00   7193.599121       -3.107277      BTC-USD
31    2020-01-01 00:00:00+00:00   7200.174316       -1.663673      BTC-USD
32    2020-01-02 00:00:00+00:00   6985.470215       -4.571976      BTC-USD
33    2020-01-03 00:00:00+00:00   7344.884277        1.280325      BTC-USD
34    2020-01-04 00:00:00+00:00   7410.656738       -0.505496      BTC-USD
...                         ...           ...             ...          ...
1799  2024-11-03 00:00:00+00:00  68741.117188       10.752234      BTC-USD
1800  2024-11-04 00:00:00+00:00  67811.507812        9.214951      BTC-USD
1801  2024-11-05 00:00:00+00:00  69359.562500       10.411841      BTC-USD
1802  2024-11-06 00:00:00+00:00  75639.078125       21.534603      BTC-USD
1803  2024-11-07 00:00:00+00:00  75904.859375       22.167156      BTC-USD

[1774 rows x 4 columns]



/tmp/ipykernel_81090/8477623.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  tickers[ticker] = tickers[ticker].drop(['Adj Close', 'High', 'Low', 'Open', 'Volume'], axis=1)


In [2]:
#Generate color palette from magma palette, excluding dark colors
from bokeh.palettes import Magma256
color_codes = [int(num) for num in np.linspace(100,255, len(ticker_names))]
colors = {k: v for k, v in zip(ticker_names, [Magma256[color] for color in color_codes])}
colors

{'BTC-USD': '#892881',
 'SOL-USD': '#C63C73',
 'ETH-USD': '#F56C5B',
 'GC=F': '#FEB57C',
 'SI=F': '#FBFCBF'}

In [ ]:
import numpy as np
from bokeh.core.properties import field
from bokeh.plotting import figure, output_file, show, column, row
from bokeh.io import curdoc
from bokeh.models import HoverTool, ColumnDataSource, RangeTool, VSpan, LabelSet, CustomJS, Legend

#Set dark theme to fit with magma palette
curdoc().theme='dark_minimal'
curdoc().title = "Interactive crypto and precious metal stock price viewer"
#The file to save the model
output_file("test.html")

#Instantiate figure object
dates = np.array(tickers['BTC-USD']['Date'], dtype=np.datetime64)
graph = figure(height=500, width=1000, x_axis_type = "datetime", 
               tools='tap,xpan,reset', toolbar_location=None, x_axis_location = "above", x_range =(dates[-365], dates[-1])) 

select = figure(title="Drag the middle and edges of the selection box to change the range",
                height=150, width=1000, y_range=graph.y_range, x_axis_type="datetime",
                tools="", toolbar_location=None)

absolute_graph = figure(height=500, width=1000, x_axis_type = "datetime", 
               tools='tap,xpan,reset', toolbar_location=None, x_axis_location = "above", x_range =graph.x_range) 


range_tool = RangeTool(x_range=graph.x_range)
range_tool.overlay.fill_color = "#900C3F"
range_tool.overlay.fill_alpha = 0.2


#Names axes
graph.xaxis.axis_label = 'Date'
graph.yaxis.axis_label = '30-day price change (%)'
absolute_graph.xaxis.axis_label = 'Date'
absolute_graph.yaxis.axis_label = 'Price ($)'

#Plotting the line graph

renderers = []
select_renderers = []
absolute_renderers = []
legend_it = []
#Adding prices
for ticker_name in ticker_names:

    source = ColumnDataSource(tickers[ticker_name])
    color = colors[ticker_name]
    l = graph.line('Date', 'RelativeChange', source=source, color=color)
    select_l= select.line('Date', 'RelativeChange', source=source, color=color)
    absolute_l = absolute_graph.line('Date', 'Close', source=source, color=color)

    renderers.append(l)
    select_renderers.append(select_l)
    absolute_renderers.append(absolute_l)

    legend_it.append((ticker_name, [l, select_l, absolute_l]))

#Adding vspans for historical events
date_strings = ["2019-12-31", "2020-03-11", "2021-02-08", "2021-09-14", "2022-02-24", "2023-03-10", "2023-10-07", "2024-11-05"]
datetime_array = np.array(date_strings, dtype=np.datetime64)
label_strings = ["COVID-19 identified", "COVID-19 declared pandemic", "Tesla invests $1.5B in BTC", "SOL worldwide outage", "Russia-Ukraine War begins", "Silicon Valley Bank Collapse", "Israel-Hamas conflicts begin", "US Election Day"]

vspan_df = pd.DataFrame(dict(dates=datetime_array, width=np.full(len(datetime_array),2), labels=label_strings))
vspan_source = ColumnDataSource(vspan_df)

vspan = VSpan(x=field("dates"), line_width=field("width"), line_color="white", line_dash="dashed")

myMax = -10000
myMin = 10000

for ticker_name in tickers:
    currentMax = tickers[ticker_name].tail(365)['RelativeChange'].max()
    currentMin = tickers[ticker_name].tail(365)['RelativeChange'].min()
    if myMax < currentMax:
        myMax = currentMax
    
    if myMin > currentMin:
        myMin = currentMin

graph.y_range.start = myMin - (abs(myMin)*0.15)
graph.y_range.end = myMax + (abs(myMax)*0.15)


labels = LabelSet(x="dates", y=graph.y_range.end*0.9, text="labels", x_offset=5, source=vspan_source, text_color="white")
graph.add_glyph(vspan_source, vspan)
select.add_glyph(vspan_source,vspan)
absolute_graph.add_glyph(vspan_source,vspan)
graph.add_layout(labels)

#Adding callback to move labels when y axis changes
label_position_callback = CustomJS(args=dict(labels=labels, graph=graph), code="""
    var y_end = graph.y_range.end
    var new_y = 0.9 * y_end
    labels.y = new_y       
""")
graph.y_range.js_on_change("end", label_position_callback)

#Adding callback to rescale y-axis to fit current timeframe for both relative and and absolute prices
y_axis_rescale_callback = CustomJS(args=dict(renderers = renderers, absolute_renderers=absolute_renderers, tickers=tickers, graph=graph, absolute_graph = absolute_graph), code="""
        let myMax = -10000
        let myMin = 100000 
        let myAbsoluteMax = 0
        let myAbsoluteMin = 1000000
        let start = graph.x_range.start
        let end = graph.x_range.end
                                   
        for (let i = 0; i < renderers.length; i++){
            var contains = renderers[i].visible

            if (contains){
                let [_, value] = Object.entries(tickers)[i]
                let extractedValues = []
                let AbsoluteExtractedValues = []
                
                for (let j = 0; j < value.length; j+=4){
                    if (value[j] >= start && value[j] <= end ){
                        extractedValues.push(value[j+2])
                        AbsoluteExtractedValues.push(value[j+1])            
                    }                   
                }

                let currentMax = Math.max(...extractedValues)
                let currentMin = Math.min(...extractedValues)
                let currentAbsoluteMax = Math.max(...AbsoluteExtractedValues)
                let currentAbsoluteMin = Math.min(...AbsoluteExtractedValues)
                if (myMax < currentMax){
                    myMax = currentMax
                }
                if (myMin > currentMin){
                    myMin = currentMin
                }
                if (myAbsoluteMax < currentAbsoluteMax){
                    myAbsoluteMax = currentAbsoluteMax
                }
                if (myAbsoluteMin > currentAbsoluteMin){
                    myAbsoluteMin = currentAbsoluteMin
                }
            }
    
        }
        console.log("myMax is: " +myMax)
        graph.y_range.end = myMax + (Math.abs(myMax) * 0.15)
        graph.y_range.start = myMin - (Math.abs(myMin) * 0.15)
                                   
        absolute_graph.y_range.end = myAbsoluteMax + (Math.abs(myAbsoluteMax) * 0.15)
        absolute_graph.y_range.start = myAbsoluteMin - (Math.abs(myAbsoluteMin) * 0.15)
                        
""")
graph.x_range.js_on_change("end", y_axis_rescale_callback)
for i in range(len(renderers)):
    renderers[i].js_on_change('visible', y_axis_rescale_callback)


#Adding miscellaneous stylization
legend = Legend(
    items=legend_it, 
    location='center',
    orientation='vertical',
    inactive_fill_color = '#383838',
    inactive_fill_alpha = 0.5
    )
legend.click_policy='hide'
graph.add_layout(legend, 'right')
absolute_graph.add_layout(legend,'right')

hover = HoverTool(tooltips=[("Asset name", "@Legend_label"),("Date", "@Date{%F}"), ("Price change (%)", "@RelativeChange{0.00}")],
                   renderers=renderers, formatters={'@Date': 'datetime'})

absolute_hover = HoverTool(tooltips=[("Asset name", "@Legend_label"),("Date", "@Date{%F}"), ("Price ($)", "@Close{0.00}")],
                   renderers=absolute_renderers, formatters={'@Date': 'datetime'})

graph.add_tools(hover)
absolute_graph.add_tools(absolute_hover)
select.ygrid.grid_line_color=None
select.add_tools(range_tool)

show(column(graph,select,absolute_graph))

/tmp/ipykernel_81090/187080012.py:14: UserWarning: no explicit representation of timezones available for np.datetime64
  dates = np.array(tickers['BTC-USD']['Date'], dtype=np.datetime64)
